In [ ]:
import Pkg
Pkg.activate("elementifinitiunipv_pkg")

1+1

Le variabili in julia vengono passate alle funzioni con il meccanismo "pass by sharing": l'oggetto non viene copiato ma gli argomenti della funzione diventano nuovi bindings e modifico l'argomento originale. 
- Se il valore è immutable (Stringhe, numeri...) creo un nuovo oggetto senza modificare l'originale
- Se il valore è mutable (per esempio array o dizionari) le modifiche fatte ad elementi degli array dentro la funzione le vediamo anche fuori dalla funzione. 

In matlab quando entravo nella funzione veniva fatta una copia dei parametri e poi ci si lavorava. Qui il contrario: quello che succede nella funzione succede anche fuori.





In [ ]:
function f(x,y)
    x[1] = 42
    y = y+7
    return y
end

a = [4,5,6]
b = 3
z = f(a,b)
println(a)
println(b)
println(z)

vantaggio rispetto a matlab che copiava tutto: risparmio tempo e memoria, magari devo fare delle piccole modifiche. spesso, per ricordarci che la funzione modifica gli argomenti, è utile scrivere function f!(x,y) ... invece che function f(x,y)....



esempio: in matlab per plottare una cosa la sintassi base è: plot(x,y).
Se voglio aggiungere qualcosa allora metto plot(x,y, "Linestyle"=.... , ....)
In julia ci sono gli argomenti optionali e keyword. 


In [ ]:
function hello(name = "gioia")  #qui name è argomento opzionale
    println("ciao, ", name, "!")
end

hello()
hello("Carlo")

Quando ho tanti tanti argomenti le cose in matlab diventano un po' più complicate ma in julia è tutto più carino: prima si mettono le variabili obbligatorie senza nulla, e quelle opzionali con la keyword associata. 
attenzione: scrivendo la funzione separo le var obbl e quelle opzionali con un ; .

In [ ]:
function describe_person(name; age = 20, city="Roma")
    println("$name ha $age anni ed è di $city")
end

Esercizio 1: funzione example che:
- accetta x obbligatorio
- parametro opzionale con valore 10
- parametro con keyword con valore predef 20

restituisce somma x+y+z


nota: "return" non è obbligatorio (ma è molto consigliato), se non lo metto la funzione restituisce la cosa che c'è alla fine. 
se non voglio un output (tipo none di python) scrivo return nothing


Differenze finite
(vedi pdf)